In [2]:
# IMPORTS
import netCDF4 as nc
import numpy as np
import datetime
from pythermalcomfort.models import utci
import matplotlib.pyplot as plt

path = "/Users/milan/git/AthleticsChampionshipsHeat/data/"

In [4]:
# OPTIONS
filename = "berlin2018fc"
utc = 2
tstart = datetime.datetime(2018,8,6,6)     # start day of the event
tend = datetime.datetime(2020,8,12,23)     # end day of the event

In [92]:
# LOAD DATA
ncfile = nc.Dataset(path+filename+".nc")
time_hours = ncfile.variables["time"][:]    # time in hours since 1900-1-1
lats = ncfile.variables["latitude"][:]      # latitudes
lons = ncfile.variables["longitude"][:]     # longitudes
ens = ncfile.variables["number"][:]

T = ncfile.variables["t2m"][:]-273.15       # 2m temperature [°C]
Td = ncfile.variables["d2m"][:]-273.15      # 2m dew point temperature [°C]
u = ncfile.variables["u10"][:]              # 10m wind u-component [m/s]
v = ncfile.variables["v10"][:]              # 10m wind v-component [m/s]

# # daily accumulated radiation fluxes
ssr_acc = ncfile.variables["ssr"][:]        # net shortwave radiation [J/m^2]
str_acc = ncfile.variables["str"][:]        # net longwave radiation [J/m^2]
ssrd_acc = ncfile.variables["ssrd"][:]      # downward shortwave radiation [J/m^2]
strd_acc = ncfile.variables["strd"][:]      # downward longwave radiation [J/m^2]
fdir_acc = ncfile.variables["fdir"][:]          # downward direct shortwave radiation [J/m^2]

ncfile.close()

In [64]:
## convert time
t0 = datetime.datetime(1900,1,1)              # netCDF is in hours since
utc_td = datetime.timedelta(hours=utc)        # convert utc from int to timedelta
time_utc = [t0+datetime.timedelta(hours=np.float64(i)) for i in time_hours]   # time in utc
time = [t+utc_td for t in time_utc]                                           # local time

julianday0 = datetime.datetime(tstart.year,1,1)                            # beginning of year
juliandays = np.array([(t-julianday0).days for t in time_utc])             # time in days since beginning of year
hours = np.array([(t-julianday0).seconds/3600 for t in time_utc])          # time in hours since beginning of year
hours_since_start = np.array([(t-tstart).days*24 + (t-tstart).seconds/3600 for t in time])

In [120]:
## relative humidity from dew point
# Magnus formula
A1 = 17.625
B1 = 243.04
rh = 100*(np.exp(A1*Td / (B1+Td)) / np.exp(A1*T / (B1+T)))

## wind speed
wspd = np.sqrt(u**2 + v**2).clip(0.5,17)

## convert radiation from accumulated J/m^2 to W/m^2
# call strr as str is string func
one_hour = 3600
ssr = ((ssr_acc[1:,:,:,:]-ssr_acc[:-1,:,:,:]).T/(hours_since_start[1:]-hours_since_start[:-1])).T/one_hour
strr = ((str_acc[1:,:,:,:]-str_acc[:-1,:,:,:]).T/(hours_since_start[1:]-hours_since_start[:-1])).T/one_hour
ssrd = ((ssrd_acc[1:,:,:,:]-ssrd_acc[:-1,:,:,:]).T/(hours_since_start[1:]-hours_since_start[:-1])).T/one_hour
strd = ((strd_acc[1:,:,:,:]-strd_acc[:-1,:,:,:]).T/(hours_since_start[1:]-hours_since_start[:-1])).T/one_hour
fdir = ((fdir_acc[1:,:,:,:]-fdir_acc[:-1,:,:,:]).T/(hours_since_start[1:]-hours_since_start[:-1])).T/one_hour

In [121]:
## radiation decomposition
# ssrd is diffusive+direct radiation, subtract the direct fdir to obtain diffusive only
ssrd_diffusive = ssrd - fdir

# ssrd is downward, ssr is net (down+up), subtract for upward only
ssru = ssrd - ssr

# strd is downward, str is net (down+up), subtract for updward only
stru = strd - strr

## solar coordinates (see Di Napoli, 2020 for more information)
def cosd(x):
    return np.cos(np.pi*x/180)
    
def sind(x):
    return np.sin(np.pi*x/180)

def tand(x):
    return np.tan(np.pi*x/180)
    
def acosd(x):
    return 180/np.pi*np.arccos(x)

def g(JD,hr):
    # angular fraction of the year in degrees
    # JD: Julian day number 0 for January 1st, 0 UTC
    # hr: hour of day (UTC)
    return 360/365.25*(JD+hr/24)

def h(hr,lambdaa,TC):
    # solar hour angle (local), NOAA 1997
    # hr: hour of day (UTC)
    # lambdaa: longitude (degrees)
    # TC: time correction
    return (hr-12)*15 + lambdaa + TC

def TC(g):
    # time correction, NOAA 1997
    return 0.004297 + 0.107029*cosd(g) + \
        -1.837877*sind(g) - 0.837378*cosd(2*g) + \
        -2.340475*sind(2*g)

def delta(g):
    # solar declination angle, Spencer 1971
    return 180/np.pi*(0.006918-0.399912*cosd(g) +
        0.070257*sind(g) - 0.006758*cosd(2*g) +
        0.000907*sind(2*g) - 0.002697*cosd(3*g) +
        0.001480*sind(3*g))
        
        
time_g = g(juliandays,hours)
declination = delta(time_g)
time_correction = TC(time_g)
hour_angle = np.array([h(hours,lon,time_correction) for lon in lons])

# hour_angle_sunrise
cos_h0 = np.array([-tand(declination)*tand(lat) for lat in lats])

In [122]:
# clip hour angles to sunrise-sunset
hminmax = np.zeros_like(fdir_acc[:,0,:,:])
nosun = np.zeros_like(fdir_acc[:,0,:,:])

for ilat,_ in enumerate(lats):
    for ilon,_ in enumerate(lons):
        for i,cos_h0i in enumerate(cos_h0[ilat,:]): 
            if hour_angle[ilon,i] < -acosd(cos_h0i):    # before sunrise
                hminmax[i,ilat,ilon] = -acosd(cos_h0i)
                nosun[i,ilat,ilon] = 1
            elif hour_angle[ilon,i] > acosd(cos_h0i):   # after sunset
                hminmax[i,ilat,ilon] = acosd(cos_h0i)
                nosun[i,ilat,ilon] = 1
            else:                                       # in between
                hminmax[i,ilat,ilon] = hour_angle[ilon,i]

# solar zenith angle
cos_phi0 = np.zeros_like(fdir[:,0,:,:])     # integration over timestep
cos_phi = np.zeros_like(fdir_acc[:,0,:,:])               

for ilat,lat in enumerate(lats):
    for ilon,_ in enumerate(lons):
        for i,decl in enumerate(declination[:-1]):
            # no sun in the time interval
            if nosun[i,ilat,ilon] == 1 and nosun[i+1,ilat,ilon] == 1:
                cos_phi0[i,ilat,ilon] = 0
            else:
                cos_phi0[i,ilat,ilon] = sind(decl)*sind(lat) + \
            1/(2*np.pi/360*(hminmax[i+1,ilat,ilon]-hminmax[i,ilat,ilon]))* \
            cosd(decl)*cosd(lat)*(sind(hminmax[i+1,ilat,ilon])-
                                    sind(hminmax[i,ilat,ilon]))
            
            cos_phi[i,ilat,ilon] = sind(decl)*sind(lat) + \
                                cosd(decl)*cosd(lat)*cosd(hour_angle[ilon,i])

## elevation angle and projection factor
elevation_angle = 90-acosd(cos_phi[1:,:,:])
fp = 0.308*cosd(elevation_angle*(0.998-elevation_angle**2/50000))

In [123]:
## COMPUTE MRT
Istar = np.zeros_like(fdir)

for i in range(len(time_utc)-1):
    for j in range(len(lons)):
        for k in range(len(lats)):
            if cos_phi0[i,j,k] > 0:
                # convert to W/m^2 from J/m^2
                if cos_phi[i,j,k] <= 0:
                    Istar[i,:,j,k] = 0.0
                else:
                    Istar[i,:,j,k] = fdir[i,:,j,k]/cos_phi0[i,j,k]/one_hour

In [124]:
alpha_ir = 0.7      # absorption coefficient of a human body
eps_p = 0.97        # emissivity coefficient of a human body
sigma = 5.67e-8     # stefan-boltzmann constant
fa = 0.5            # angle factors
fpIstar = np.zeros_like(Istar)
for i in range(len(ens)):
    fpIstar[:,i,:,:] = fp*Istar[:,i,:,:]

MRT = ((fa*strd + fa*stru + alpha_ir/eps_p*(fa*(ssrd_diffusive+ssru)+fpIstar))/sigma)**0.25-273.15

In [125]:
MRT

masked_array(
  data=[[[[35.3202939 , 35.24847609, 34.97593959, 34.67679008,
           34.80606035],
          [33.95760024, 33.83370364, 33.64965515, 33.8132709 ,
           34.09844076],
          [33.81598066, 33.71478501, 33.36257441, 33.61334832,
           33.85076865],
          [36.18970774, 36.08979345, 36.06507166, 35.58048872,
           35.13009881],
          [39.4053976 , 39.06603744, 38.61637956, 37.74621963,
           36.8459112 ]],

         [[36.08161537, 35.91221872, 35.29246615, 34.68976514,
           35.0573949 ],
          [35.59917813, 35.18208763, 35.04477754, 35.01352119,
           35.55832553],
          [35.39735665, 35.26961782, 34.93358632, 35.24477235,
           35.85934078],
          [35.45144735, 35.69510033, 35.85584993, 35.56735547,
           35.40480285],
          [36.52329247, 35.95561968, 36.08326234, 35.52602487,
           35.16876503]],

         [[42.59328086, 42.17959836, 40.86781122, 39.86466369,
           39.16614875],
          [42.

In [126]:
## COMPUTE UTCI
UTCI = np.zeros_like(MRT)        # UTCI
UTCIwind = np.zeros_like(MRT)    # UTCI with wind varying only, other variables default
UTCIrh = np.zeros_like(MRT)      # UTCI with relative humidity only, other variables default
UTCImrt = np.zeros_like(MRT)     # UTCI with radiation only, other variables default

for i in range(UTCI.shape[0]):
    for j in range(UTCI.shape[1]):
        for k in range(UTCI.shape[2]):
            for l in range(UTCI.shape[3]):
                UTCI[i,j,k,l] = utci(T[i+1,j,k,l],MRT[i,j,k,l],wspd[i+1,j,k,l],rh[i+1,j,k,l])
                UTCIwind[i,j,k,l] = utci(T[i+1,j,k,l],T[i+1,j,k,l],wspd[i+1,j,k,l],50)
                UTCIrh[i,j,k,l] = utci(T[i+1,j,k,l],T[i+1,j,k,l],0.5,rh[i+1,j,k,l])
                UTCImrt[i,j,k,l] = utci(T[i+1,j,k,l],MRT[i,j,k,l],0.5,50)

In [127]:
## SAVE UTCI
np.savez(path+filename+"utci.npz",time=time,UTCI=UTCI,UTCIwind=UTCIwind,UTCIrh=UTCIrh,UTCImrt=UTCImrt)

In [108]:
# preallocate
ndays = (tend-tstart).days + 1
UTCI_dailymax = np.zeros(ndays)      # in the central grid box
UTCI_dailymean = np.zeros(ndays)
UTCI_3daymean = np.zeros(ndays)
UTCI_dailydiff = np.zeros(ndays)

# uncertainty
UTCI_dailymax0 = np.zeros(ndays)     # lowest max value in any grid box
UTCI_dailymax1 = np.zeros(ndays)     # highest max value in any grid box

UTCI_dailymean0 = np.zeros(ndays)    # highest mean value in any grid box
UTCI_dailymean1 = np.zeros(ndays)    # lowest max value in any grid box

UTCI_3daymean0 = np.zeros(ndays)     # lowest 3day mean value in any grid box
UTCI_3daymean1 = np.zeros(ndays)     # highest 3day mean value in any grid box

UTCI_dailydiff0 = np.zeros(ndays)    # lowest daily difference in mean UTCI in any grid box
UTCI_dailydiff1 = np.zeros(ndays)    # highest daily difference in mean UTCI in any grid box

In [109]:
# UTCI daily maximum and mean
for i in range(ndays):
    # pick data from respective day 9:00 to 21:00 local time
    itd = datetime.timedelta(days=i)
    iday = [t.date() == (tstart+itd).date() and t.hour > 8 and t.hour < 22 for t in time[1:]]
    UTCI_dailymax[i] = np.max(UTCI[iday,2,2])
    UTCI_dailymean[i] = np.mean(UTCI[iday,2,2])
    
    UTCI_dailymax0[i] = np.min(np.max(UTCI[iday,:,:],axis=0))
    UTCI_dailymax1[i] = np.max(np.max(UTCI[iday,:,:],axis=0))
    
    UTCI_dailymean0[i] = np.min(np.mean(UTCI[iday,:,:],axis=0))
    UTCI_dailymean1[i] = np.max(np.mean(UTCI[iday,:,:],axis=0))
    
    # 3-day mean
    # pick data from days i,i-1,i-2 at 9:00 to 21:00
    itd0 = datetime.timedelta(days=i-2)
    
    iday = [t.date() <= (tstart+itd).date() and
            t.date() >= (tstart+itd0).date() and
            t.hour > 8 and t.hour < 22 for t in time[1:]]
    
    UTCI_3daymean[i] = np.mean(UTCI[iday,2,2])
    UTCI_3daymean0[i] = np.min(np.mean(UTCI[iday,:,:],axis=0))
    UTCI_3daymean1[i] = np.max(np.mean(UTCI[iday,:,:],axis=0))
    
    # difference daily mean UTCI to previous day
    iday = [t.date() == (tstart+itd).date() and t.hour > 8 and t.hour < 22 for t in time[1:]]
    itd1 = datetime.timedelta(days=i-1)
    iprevday = [t.date() == (tstart+itd1).date() and t.hour > 8 and t.hour < 22 for t in time[1:]]
    
    UTCI_dailydiff[i] = np.mean(UTCI[iday,2,2]) - np.mean(UTCI[iprevday,2,2])
    UTCI_dailydiff0[i] = np.min(np.mean(UTCI[iday,:,:],axis=0) - np.mean(UTCI[iprevday,:,:],axis=0))
    UTCI_dailydiff1[i] = np.max(np.mean(UTCI[iday,:,:],axis=0) - np.mean(UTCI[iprevday,:,:],axis=0))
    

In [110]:
UTCI_dailymax

array([33.6, 36.5, 37.1, 36.8, 25.6, 21.6, 29.6])

In [111]:
# SAVE TO NPZ
np.savez(path+filename+"utci_daily.npz",UTCI_dailymax=UTCI_dailymax,
                                         UTCI_dailymean=UTCI_dailymean,
                                         UTCI_3daymean=UTCI_3daymean,
                                         UTCI_dailydiff=UTCI_dailydiff,
                                         UTCI_dailymax0=UTCI_dailymax0,
                                         UTCI_dailymax1=UTCI_dailymax1,
                                         UTCI_dailymean0=UTCI_dailymean0,
                                         UTCI_dailymean1=UTCI_dailymean1,
                                         UTCI_3daymean0=UTCI_3daymean0,
                                         UTCI_3daymean1=UTCI_3daymean1,
                                         UTCI_dailydiff0=UTCI_dailydiff0,
                                         UTCI_dailydiff1=UTCI_dailydiff1)

In [112]:
# SAVE TO CSV
fileout = open(path+"csv/"+filename+"utci_daily.csv","w")
fileout.write("Date,Day,Daily maximum UTCI [˚C],Daily mean UTCI [˚C],3-day mean UTCI [˚C],daily difference mean UTCI [˚C]\n")
for i in range(ndays):
    itd = datetime.timedelta(days=i)
    s = ",".join([str((tstart+itd).date()),
                  "{}".format(i+1),
                  "{:.4f}".format(UTCI_dailymax[i]),
                  "{:.4f}".format(UTCI_dailymean[i]),
                  "{:.4f}".format(UTCI_3daymean[i]),
                  "{:.4f}".format(UTCI_dailydiff[i])])
    fileout.write(s+"\n")

fileout.close()